파일 불러오기(인코딩이 다름..)

In [2]:
import pandas as pd

# CSV 불러오기
# df = pd.read_csv("/data1/workspace/gold_standardized.csv", encoding="utf-8")  # 필요시 cp949, euc-kr 등으로 변경

# 수정된 코드
df = pd.read_csv('/data1/workspace/prepro_calcul/t4.csv', encoding='cp949')
# 또는
# df = pd.read_csv('파일명.csv', encoding='euc-kr')

In [3]:
print(df.head())

                                               title  \
0  wee1 confers resistance to krasg12c inhibitors...   
1  palbociclib overcomes afatinib resistance in n...   
2  regorafenib suppresses epidermal growth factor...   
3  her2 exon 20 insertions in non-small-cell lung...   
4  xie-bai-san increases nsclc cells sensitivity ...   

                                                drug  \
0          ['sotorasib', 'adavosertib', 'puromycin']   
1           ['palbociclib', 'letrozole', 'afatinib']   
2  ['regorafenib', 'erlotinib', '6bromo(2chloroph...   
3  ['pyrotinib', 'afatinib', 'trastuzumabemtansine']   
4                         ['sirolimus', 'gefitinib']   

                                          normalized  
0            ['sotorasib','adavosertib','puromycin']  
1             ['palbociclib','letrozole','afatinib']  
2  ['regorafenib','erlotinib','6bromo(2chlorophen...  
3    ['pyrotinib','afatinib','trastuzumabemtansine']  
4                          ['sirolimus','gefitinib'

In [4]:
# CSV 불러오기
df1 = pd.read_csv("/data1/workspace/prepro_calcul/t4.csv", encoding="utf-8")  # 필요시 cp949, euc-kr 등으로 변경

print(df1.head())

                                               title  \
0  wee1 confers resistance to krasg12c inhibitors...   
1  palbociclib overcomes afatinib resistance in n...   
2  regorafenib suppresses epidermal growth factor...   
3  her2 exon 20 insertions in non-small-cell lung...   
4  xie-bai-san increases nsclc cells sensitivity ...   

                                                drug  \
0          ['sotorasib', 'adavosertib', 'puromycin']   
1           ['palbociclib', 'letrozole', 'afatinib']   
2  ['regorafenib', 'erlotinib', '6bromo(2chloroph...   
3  ['pyrotinib', 'afatinib', 'trastuzumabemtansine']   
4                         ['sirolimus', 'gefitinib']   

                                          normalized  
0            ['sotorasib','adavosertib','puromycin']  
1             ['palbociclib','letrozole','afatinib']  
2  ['regorafenib','erlotinib','6bromo(2chlorophen...  
3    ['pyrotinib','afatinib','trastuzumabemtansine']  
4                          ['sirolimus','gefitinib'

예측셋 정규화

In [6]:
import pandas as pd

# 1. CSV 읽기
# expert_df = pd.read_csv("/data1/workspace/gold_standardized _doublecheck.csv", encoding='cp949')
llm_df = pd.read_csv("/data1/workspace/prepro_calcul/t4.csv", encoding="utf-8")

# 2. 정규화 함수
def normalize_drug_list(drug_str):
    if pd.isna(drug_str):
        return ""
    drugs = drug_str.split(';')
    # 하이픈, 공백 제거 + 소문자화
    drugs = [d.replace('-', '').replace(' ', '').lower().strip() for d in drugs if d.strip() != '']
    # 리스트 대신 문자열로 반환
    return ';'.join(drugs)

# 3. 정규화 적용
# expert_df['normalized'] = expert_df['single_drug'].apply(normalize_drug_list)
llm_df['normalized'] = llm_df['drug'].apply(normalize_drug_list)

# 4. 저장 (UTF-8)
# expert_df.to_csv("expert_drugs_utf8.csv", encoding='utf-8', index=False)
llm_df.to_csv("/data1/workspace/prepro_calcul/t4.csv", encoding='utf-8', index=False)


In [ ]:
# print(expert_df.head())

In [ ]:
print(llm_df.head())

                                                drug  \
0          ['sotorasib', 'adavosertib', 'puromycin']   
1           ['palbociclib', 'letrozole', 'afatinib']   
2  ['regorafenib', 'erlotinib', '6bromo(2chloroph...   
3  ['pyrotinib', 'afatinib', 'trastuzumabemtansine']   
4                         ['sirolimus', 'gefitinib']   

                                               title  \
0  wee1 confers resistance to krasg12c inhibitors...   
1  palbociclib overcomes afatinib resistance in n...   
2  regorafenib suppresses epidermal growth factor...   
3  her2 exon 20 insertions in non-small-cell lung...   
4  xie-bai-san increases nsclc cells sensitivity ...   

                                   Generic_Drug_List  \
0                    SOTORASIB;ADAVOSERTIB;PUROMYCIN   
1                     PALBOCICLIB;LETROZOLE;AFATINIB   
2  REGORAFENIB;ERLOTINIB;6-BROMO-(2-CHLOROPHENYL)...   
3           PYROTINIB;AFATINIB;TRASTUZUMAB EMTANSINE   
4                                SIROLIMUS;GEF

빈리스트 제거

In [8]:
import ast
import pandas as pd
import numpy as np

def list_or_literal_to_str(x, delimiter=';'):
    # NaN 처리
    if pd.isna(x):
        return ""
    # 이미 리스트인 경우
    if isinstance(x, list):
        cleaned = [str(i).strip() for i in x if str(i).strip() != ""]
        return delimiter.join(cleaned)
    # 문자열인 경우: "a;b" 은 그대로, "['a', 'b']" 같은 리터럴이면 파싱
    if isinstance(x, str):
        s = x.strip()
        # 빈 문자열
        if s == "":
            return ""
        # 이미 세미콜론으로 구분된 정상 문자열이면 그대로 (단 공백 정리)
        if delimiter in s:
            parts = [p.strip() for p in s.split(delimiter) if p.strip() != ""]
            return delimiter.join(parts)
        # 문자열 리터럴 형태인지 시도 파싱 (예: "['a','b']" or "['a', 'b']")
        try:
            lit = ast.literal_eval(s)
            if isinstance(lit, (list, tuple)):
                cleaned = [str(i).strip() for i in lit if str(i).strip() != ""]
                return delimiter.join(cleaned)
        except Exception:
            pass
        # 그 외 일반 문자열: 공백 제거 후 소문자화(필요시)
        return s.strip()
    # 그 외 타입은 문자열로 변환
    return str(x).strip()

# 예: expert_df와 llm_df에 적용
# expert_df['normalized'] = expert_df['normalized'].apply(list_or_literal_to_str)
llm_df['normalized'] = llm_df['normalized'].apply(list_or_literal_to_str)

# 확인용 출력 (상위 5개)
print("=== expert_df normalized sample ===")
# print(expert_df['normalized'].head())
print("=== llm_df normalized sample ===")
print(llm_df['normalized'].head())

# 필요하면 바로 semicolon 기준으로 다시 리스트로 비교할 때:
# expert_df['normalized_list'] = expert_df['normalized'].apply(lambda s: [] if s=="" else [i.strip() for i in s.split(';')])
llm_df['normalized_list'] = llm_df['normalized'].apply(lambda s: [] if s=="" else [i.strip() for i in s.split(';')])


=== expert_df normalized sample ===
=== llm_df normalized sample ===
0                      sotorasib;adavosertib;puromycin
1                       palbociclib;letrozole;afatinib
2    regorafenib;erlotinib;6bromo(2chlorophenyl)qui...
3              pyrotinib;afatinib;trastuzumabemtansine
4                                  sirolimus;gefitinib
Name: normalized, dtype: object


In [9]:
import os

# 원본 파일 경로
FILE_PATH = "/data1/workspace/prepro_calcul/t4.csv"

# 백업 파일 경로
# BACKUP_PATH = FILE_PATH.replace(".csv", "_backup.csv")

# 1️⃣ 백업 만들기 (덮어쓰기 전 원본 보존)
# if os.path.exists(FILE_PATH):
#     os.rename(FILE_PATH, BACKUP_PATH)
#     print(f"🧾 원본 백업 완료: {BACKUP_PATH}")

# 2️⃣ 덮어쓰기 저장
# 예: expert_df에 정제 결과가 있다면
llm_df.to_csv(FILE_PATH, index=False, encoding='utf-8')

print(f"💾 t4.csv 덮어쓰기 완료: {FILE_PATH}")


💾 t4.csv 덮어쓰기 완료: /data1/workspace/prepro_calcul/t4.csv
